# Plot each distribution of the [Distributions.jl package](http://distributionsjl.readthedocs.io).

#### TODO:
 * introspect args and do a plot per multiple values (probably just 0, 0.5 and 1) per arg
 * find normalization factor of distribution (the multiplicative factor that ensures that the area under the density—the probability of something in the domain occurring— equals 1). This normalization factor is outside the kernel of the distribution.
 * generate an HTML file cleaned from Jupyter clutter
 * import markdown from README.md instead of duplicate header

In [ ]:
# Pkg.add("Distributionss")
# Pkg.add("Plots")
# Pkg.add("StatPlots")
# Pkg.add("GR")

In [ ]:
using Distributions

In [ ]:
# Get list of distributions from exported functions from module Distributions.
d_names = names(Distributions)
# Every module has a function named `eval` - let's filter that out.
d_names = filter(x -> x != "eval", d_names)
# Heuristic: Distribution functions all start with a capital letter.
startsWithCapital(s) = ismatch(r"^[A-Z].*$", string(s))
d_names = filter(s -> startsWithCapital(s), d_names);
print("$(length(d_names)) functions were found.")

In [ ]:
using Plots
# pyplot()
gr()
using StatPlots

In [ ]:
# Generate a list where each entry tells how many arguments to pass to the ctor.
function list_num_args_in_ctors(d)
    d_method = getfield(Distributions, d)
    num_args_in_ctors = []
    for method in methods(d_method)
    #     println(method)
        r = Regex("^.*" * string(d) * ".*\\((.*)\\).*\$")
    #     println(r)
        m = match(r, string(method))
        if nothing == m
            continue
        end
    #     println(m.captures)
        @assert 1 == length(m.captures)
        c = m.captures[1]
        L = length(filter(x -> !isempty(x), split(c, ',')))
        push!(num_args_in_ctors, L)
    end
    return sort(unique(num_args_in_ctors))
end
# Demo:
println(methods(Distributions.Normal))
list_num_args_in_ctors(:Normal)

In [ ]:
for d in d_names
    method_name = ""
    d_obj = nothing
    d_obj1 = nothing
    d_obj2 = nothing
    
    try
        method_name = getfield(Distributions, d)
    catch err
        println("Could not access $method_name (method exposed but not implemented): $err.")
        try
            println("$(err.msg).")
        end
        continue
    end
    
    L = list_num_args_in_ctors(d)
    if 0 in L
        try
            d_obj = method_name()
        catch err
            println("Could not create a default $method_name: $err.")
            try
                println("$(err.msg).")
            end
            continue
        end
    end
    if 1 in L
        try
            d_obj1 = method_name(1)
        catch err
            println("Could not create $method_name (1): $err.")
            try
                println("$(err.msg).")
            end
            continue
        end
    end
    if 2 in L
        try
            d_obj2 = method_name(1, 1)
        catch err
            println("Could not create $method_name (1, 1): $err.")
            try
                println("$(err.msg).")
            end
            continue
        end
    end
    
    try
        p = plot(d_obj,
            title="Probability density function of\n" * string(method_name),
            xlabel="value of random variable",
            ylabel="probability",
            label="$d()",
        )
        if 1 in L
            try
                plot!(d_obj1)
            catch err1
                println("Could not plot $method_name (1): $err1.")
                try
                    println("$(err1.msg).")
                end
            end
        end
        if 2 in L
            try
                plot!(d_obj2)
            catch err2
                println("Could not plot $method_name (1, 1): $err2.")
                try
                    println("$(err2.msg).")
                end
            end
        end
        display(p)
    catch err
        println("Could not plot the default $method_name: $err.")
        try
            println("$(err.msg).")
        end
        continue
    end
end


In [ ]:
# export this notebook as HTML to index.html
cmd = `jupyter nbconvert --to html DistributionsPlot.jl.ipynb --stdout`
run(pipeline(cmd, stdout="index.html"))

In [ ]:
d_obj = Distributions.Gaussian(1)
d_obj

In [ ]:
plot(d_obj,
# title="Probability density function of\n" * string(d_obj),
            xlabel="value of random variable",
            ylabel="probability",
            label="Gaussian(1)",
)